<h1 style="color: rgba(237, 153, 29, 1);">Algoritmos de Substituição de Páginas</h1>
<h2>LRU com Envelhecimento</h2>

<small>
<p><strong>IMPORTANTE</strong>: O comando '%%file' é usado no Python para criar arquivos .java no diretório onde este notebook está salvo. Os arquivos criados são nomeados conforme o identificador fornecido após o comando '%%file'.</p>
</small>

<h3><i>Least Recently Used (LRU)</i></h3>

<p>Quando o sistema operacional usa paginação, ele mantém parte das páginas de um processo na memória principal (RAM) e o restante no disco.
Como a RAM é limitada, pode ser necessário substituir uma página (remover uma página da RAM para liberar espaço) quando uma nova precisa ser carregada.</p>
<p>Nesse contexto, o LRU é utilizado como uma política de substituição de páginas que remove da memória a página menos recentemente utilizada, ou seja, aquela que não é acessada há mais tempo. Essa política baseia-se no princípio da localidade temporal, que assume que páginas acessadas recentemente provavelmente serão utilizadas novamente em breve.</p>

<h3>O Algoritmo de Envelhecimento</h3>

<p>O LRU puro não é implementado na prática porque sua aplicação exigiria registrar e atualizar, a cada acesso à memória, a ordem exata de utilização de todas as páginas. Além disso, o conjunto de páginas na memória principal pode ser enorme, contendo milhares de quadros, o que tornaria essa atualização inviável em termos de desempenho.</p>

<p>Como alternativa, o método de <strong>envelhecimento (<i>aging</i>)</strong> surge como uma solução viável e eficiente, pois realiza atualizações em intervalos regulares (por exemplo, a cada 100 ms) utilizando bits de referência e contadores para estimar o tempo desde o último acesso a cada página.</p>

<p>O processo de envelhecimento tem duas etapas principais:</p>

<ul>
    <li><strong style="color: #E0901B;">1</strong> - Deslocamento de bits (envelhecimento): Periodicamente, o contador de cada página é deslocado um bit para a direita. Isso faz com que o <strong>peso</strong> das referências antigas diminua, efetivamente <strong>envelhecendo</strong> a página.</li>
    <li><strong style="color: #E0901B;">2</strong> - Atualização no acesso: Toda vez que uma página é referenciada (lida ou escrita), o bit mais significativo (<i>Most Significant Bit</i> - MSB) do seu contador é definido como <strong>1</strong>.</li>
</ul>

<p>Quando uma falha de página ocorre e a memória está cheia, a página com o <strong>menor valor</strong> no contador é escolhida para ser substituída.</p>

<p>As etapas do processo de envelhecimento são apresentadas na <strong>Figura 1</strong>. Note que um bit <strong>R</strong> é utilizado para registrar as páginas que foram referenciadas ao longo do tempo. Quando a atualização dos contadores é realizada, todos os bits de cada contador são deslocados para a direita, e o bit R é inserido como o bit mais significativo do contador. Após esse processo, o bit R volta a assumir o valor zero.</p>

<figure>
    <img src="contadores.png" alt="Atualização dos contadores de páginas.">
    <figcaption>Figura 1: Atribuição do bit R seguida da atualização dos contadores de páginas.</figcaption>
</figure>

<h3>Ferramentas da Linguagem C - Manipulação de Bits</h3>

<p>Para implementar o LRU com envelhecimento, precisaremos usar operadores de bits em C. Vamos focar em dois:</p>

<ul>
    <li><strong style="color: #E0901B;">1</strong> - Deslocamento para a direita (>>): Desloca os bits de uma variável para a direita. Por exemplo, se tivermos um contador de 8 bits (um <strong>unsigned char</strong>) com o valor 132 (binário 10000100), ao deslocá-lo uma posição para a direita, ele se torna 66 (binário 01000010).</li>
    <li><strong style="color: #E0901B;">2</strong> - Operador OR (|): Quando uma página na memória é acessada novamente (um <i>hit</i>), precisamos definir seu bit mais significativo (MSB) como 1. Para um contador de 8 bits, o MSB representa o valor 128 (binário 10000000). Essa operação é realizada a partir do operador OU bit-a-bit (|). Este operador compara dois números bit a bit. Se um dos bits (ou ambos) for 1, o resultado é 1. Caso contrário, é 0.</li>
</ul>

<h4 style="color: #2d3436;"><strong>Código 1</strong>: Mecanismo de envelhecimento de contadores baseado no deslocamento de bits.</h4>

In [2]:
%%file deslocamento.c

// Comando de compilação: gcc -Wall deslocamento.c -o out
// Comando de execução: ./out

#include <stdio.h>

int main() {
    unsigned char contador = 0x84; // Decimal: 132, binário: 10000100

    printf("Valor original: %d (binário: 10000100)\n", contador);

    // Desloca 1 bit para a direita
    contador = contador >> 1; 

    printf("Novo valor: %d (binário: 01000010)\n", contador);

    return 0;
}

Writing deslocamento.c


<h4 style="color: #2d3436;"><strong>Código 2</strong>: Inserindo o valor 1 no bit mais significativo (MSB) do contador.</h4>

In [3]:
%%file operacaoOR.c

// Comando de compilação: gcc -Wall operacaoOR.c -o out
// Comando de execução: ./out

#include <stdio.h>

int main() {
    unsigned char contador = 0x42; // Decimal: 66, binário: 01000010

    printf("Valor original: %d (binário: 01000010)\n", contador);

    // Define o bit mais significativo (MSB) como 1
    contador = contador | 0x80; // Máscara de bits - decimal: 128, binário é 10000000

    printf("Novo valor: %d (binário: 11000010)\n", contador); // O resultado é 194

    return 0;
}

Overwriting operacaoOR.c
